In [1]:
import matplotlib.pyplot as plt
import numpy as np
from findfile import find_result_files
from scipy.ndimage import gaussian_filter1d
import glob
from matplotlib import colors as mcolors
import pandas as pd

file_dir='output/5/0.1'
save_dir='output/acc'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)
name= file_dir.split('/')[-1]
algorithms1=['FedKDMRV3'
             ,'FedAvg'
            ,'FedMR_'
            # ,'FedKDMRV3'
            # ,'FedKDMRV11'
            # ,'FedKDMRV12'
            # ,'FedKDMRV21'
            # ,'FedKDMRV22'
            ,'FedCodl_'
            ,'FedProx'
            # ,'FedExP'
            # ,'FedMRwG'
            # ,'FedKDMR_d'
            # ,'Fed2WKDMRV3'
            # ,'Fed2Codl_revise'
           ]
other1=['cifar100_','test_acc','pre_50','KD_0.5']
# algorithms2=['FedMR'
#             # ,'FedKDMRV21'
#              ,'FedAvg'
#              # ,'FedKDMRV3'
            
#            ]
# other2=['cifar10_','test_loss','pre_50']
file_list1=find_result_files(file_dir,algorithms1,other1)
# file_list2=find_result_files(file_dir,algorithms2,other2)
# file_list=file_list1+file_list2
file_list=file_list1
num=len(file_list)
# fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(12, 18), sharex=True)
filename = f"{other1[0].strip('_')}_{other1[1]}_{name}_{num}.pdf"
save_path = os.path.join(save_dir, filename)
order= ['FedMR','FedKDMR','FedCodl','FedAvg','FedProx']
def get_priority(filename):
    for i, key in enumerate(order):
        if key in filename:
            return i
    return len(order)
file_list.sort(key=get_priority)

algorithm_accuracies = {algo: [] for algo in order}

# 处理每个文件
for file_path in file_list:
    try:
        # 读取文件数据
        with open(file_path, 'r') as file:
            data = file.read().strip().split()
        
        # 提取精度值（跳过第一个元素）
        accuracy_values = [float(x) for x in data[1:]]
        
        # 计算当前文件的平均精度
        avg_accuracy = sum(accuracy_values) / len(accuracy_values)
        
        # 识别算法名称
        algorithm_name = None
        for algo in order:
            if algo in file_path:
                algorithm_name = algo
                break
        
        # 特殊处理 FedKDMR 算法
        if 'FedKDMR' in file_path:
            algorithm_name = 'FedKDMR'
        
        # 存储结果
        if algorithm_name and algorithm_name in algorithm_accuracies:
            algorithm_accuracies[algorithm_name].append(avg_accuracy)
    
    except Exception as e:
        print(f"处理文件 {file_path} 时出错: {e}")

# 计算每个算法的最终平均精度
final_results = {}
for algo, acc_list in algorithm_accuracies.items():
    if acc_list:
        final_results[algo] = sum(acc_list) / len(acc_list)
    else:
        final_results[algo] = None  # 没有数据的算法

# 创建结果DataFrame
csv_filename = f"{other1[0].strip('_')}.csv"
csv_path = os.path.join(save_dir, csv_filename)

# 检查CSV文件是否存在
if os.path.exists(csv_path):
    # 读取现有数据
    df = pd.read_csv(csv_path, index_col=0)
    
    # 添加新列（如果列名已存在则覆盖）
    df[name] = pd.Series(final_results)
else:
    # 创建新的DataFrame
    df = pd.DataFrame.from_dict(final_results, orient='index', columns=[name])

# 确保算法按指定顺序排列
df = df.reindex(order)

# 保存到CSV
df.to_csv(csv_path)
print(f"结果已保存到: {csv_path}")

# 打印结果
print("\n=== 平均精度结果 ===")
print(f"数据集: {other1[0].strip('_')}, 设置: {name}")
print("-" * 40)
print(df)
print("=" * 40)

ModuleNotFoundError: No module named 'findfile'